# Livelike Basic Usage Demo: PUMS Replicate Weights

- ACS 2014 - 2019 5-Year Estimates
- UP Baseline Constraints: employment, industry, occupation, education
- Knox County, TN

## Setup

In [1]:
%load_ext watermark
%watermark

Last updated: 2025-01-31T09:43:07.518572-05:00

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.31.0

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



In [2]:
import pathlib

import geopandas as gpd
import numpy as np
import pandas as pd
from likeness_vitals.vitals import get_censusapikey, match
from pymedm import PMEDM, batch, diagnostics

from livelike import acs, est, homesim, multi
from livelike.utils import clear_acs_cache

%watermark -w
%watermark -iv

/Users/41q/gitlab_repos/likeness/likeness-vitals/likeness_vitals/__init__.py:12: FutureWarning: The ``likeness-vitals`` package is transitioning to a modular API for a cleaner top-level namespace. Classes, functions, and attributes that are currently available in the top-level will only be available within their home modules. Both the current and the updated API will be in place for at least two minor releases following the inclusion of this warning. The release at the time of this warning inclusion was ``v1.3.0``, so the current API will be stable until ``v1.5.0`` at the soonest, and the ``v2.0.0`` release will be the absolute deadline. See [https://code.ornl.gov/likeness/likeness-vitals/-/issues/32] for details on how to update dependant packages.
  warnings.warn(


Watermark: 2.5.0

pymedm         : 2.1.1.dev4+g785edf4
numpy          : 2.0.2
pandas         : 2.2.3
livelike       : 1.4.2
geopandas      : 1.0.1
likeness_vitals: 1.4.3.dev6+g2e663aa



### Read Census API Key

In [3]:
key = get_censusapikey(pathlib.Path("..", ""))

### Create baseline PUMA representation

The `puma` class stores a representation of the PUMA based on ACS constraints. It contains the microdata and block group/tract estimates and standard errors required for P-MEDM.

**TODO**: It may be safer to generate 80 replicates every time, as [standard error estimation](https://usa.ipums.org/usa/repwt.shtml) relies on this. However, it's also much more computationally intensive. It's also not clear whether each synthetic population replicate really captures statistical variation in the PUMS, as P-MEDM does not preserve the sample weights. Is a bootstrapped approximation of the standard errors still valid? Need to chew on this for awhile...

In [4]:
puma = "4701602"

In [5]:
mpu = multi.make_replicate_pumas(
    puma,
    nreps=5,
    censusapikey=key,
    cache_folder="replicate_demo_cache",
)

In [6]:
mpu

{'4701602_5': <livelike.acs.puma at 0x1109bdf40>,
 '4701602_4': <livelike.acs.puma at 0x309cb4fe0>,
 '4701602_3': <livelike.acs.puma at 0x31aa42810>,
 '4701602_2': <livelike.acs.puma at 0x31a98d2e0>,
 '4701602_1': <livelike.acs.puma at 0x323747ad0>,
 '4701602_0': <livelike.acs.puma at 0x31b52f170>}

In [7]:
pmds = batch.batch_solve(mpu)

In [8]:
pmds

{'4701602_5': <pymedm.pmedm.PMEDM at 0x31aa54290>,
 '4701602_4': <pymedm.pmedm.PMEDM at 0x31a4e2480>,
 '4701602_3': <pymedm.pmedm.PMEDM at 0x31a4e0410>,
 '4701602_2': <pymedm.pmedm.PMEDM at 0x31a4e0050>,
 '4701602_1': <pymedm.pmedm.PMEDM at 0x32365fc20>,
 '4701602_0': <pymedm.pmedm.PMEDM at 0x32365e900>}

In [9]:
# objective values
for puma_replicate, pmd in pmds.items():
    print(f"{puma_replicate=}: {pmd.res.state.value}")

puma_replicate='4701602_5': -1.2863682423685434
puma_replicate='4701602_4': -1.2705805691116385
puma_replicate='4701602_3': -1.2834165796559245
puma_replicate='4701602_2': -1.281641025577804
puma_replicate='4701602_1': -1.2854840117248447
puma_replicate='4701602_0': -1.2538384403196665


### Diagnostics

As a quick diagnostic, we measure the proportion of synthetic constraint estimates fitting the ACS 90% margins of error (MOEs), or the "moe_fit_rate":

In [10]:
for p in mpu:
    mfr = diagnostics.moe_fit_rate(
        mpu[p].est_ind, mpu[p].est_g2, mpu[p].se_g2, pmds[p].almat
    )

    print("PUMA " + p + ": " + str(np.round(mfr["moe_fit_rate"], 4)))

PUMA 4701602_5: 0.9996
PUMA 4701602_4: 0.9996
PUMA 4701602_3: 0.9996
PUMA 4701602_2: 0.9996
PUMA 4701602_1: 0.9996
PUMA 4701602_0: 0.9996


## Population Synthesis

Use Lovelace and Ballas' (2013) "Truncate, Replicate, Sample" method to create 1 realization of the residential synthetic population per replicate:

In [11]:
hs = multi.synthesize_multi(mpu, pmds, nsim=1)

In [12]:
hs

,sim,geoid,count,replicate
h_id,,,,
2015000038596,0,470930049001,1,4701602_5
2015000064675,0,470930049001,2,4701602_5
2015000067973,0,470930049001,2,4701602_5
2015000079640,0,470930049001,1,4701602_5
2015000087870,0,470930049001,2,4701602_5
...,...,...,...,...
2019HU1388193,0,470930065022,1,4701602_0
2019HU1396891,0,470930065022,1,4701602_0
2019HU1400850,0,470930065022,2,4701602_0
